In [3]:
%matplotlib notebook
#Import pandas and numpy to handle data
import pandas as pd
import numpy as np

#import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Processing the data: Tokenization


In [ ]:
#Visualizing the data


In [33]:
#Step2: Create a Neural Network
#Import all libraries for creating a deep neural network
import keras
#Sequential is the standard type of neural network with stackable layers
from keras.models import Sequential
#Dense: Standard layers with every node connected, dropout: avoids overfitting
from keras.layers import Dense, Dropout, Activation;

In [5]:
#Create the model
model_nn = Sequential()